In [1]:
import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

/Users/captain2mac/Documents/Code/HuggingFace/.venv/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_4step_unet.safetensors" # Use the correct ckpt for your step setting!
checkpoints = {
    "1-Step" : ["sdxl_lightning_1step_unet_x0.safetensors", 1],
    "2-Step" : ["sdxl_lightning_2step_unet.safetensors", 2],
    "4-Step" : ["sdxl_lightning_4step_unet.safetensors", 4],
    "8-Step" : ["sdxl_lightning_8step_unet.safetensors", 8],
}

In [3]:
device = 'mps'
torch.ones(2, device=device)

tensor([1., 1.], device='mps:0')

Unet

In [4]:
unet = UNet2DConditionModel.from_config(base, subfolder='unet').to(device)
ckpt_file = hf_hub_download(repo, ckpt)
unet_state_dict = load_file(ckpt_file, device=device)
unet.load_state_dict(unet_state_dict)

/Users/captain2mac/Documents/Code/HuggingFace/.venv/lib/python3.9/site-packages/diffusers/configuration_utils.py:244: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'diffusers.models.unets.unet_2d_condition.UNet2DConditionModel'>.load_config(...) followed by <class 'diffusers.models.unets.unet_2d_condition.UNet2DConditionModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)


<All keys matched successfully>

SDXL pipeline loading

In [5]:
pipeline = StableDiffusionXLPipeline.from_pretrained(base, unet=unet, torch_dtype=torch.float16, variant='fp16').to(device)

pipeline.scheduler = EulerDiscreteScheduler.from_config(pipeline.scheduler.config, timestep_spacing="trailing")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Inferencing!

In [6]:
pipeline("A porche 911 driving into the sunset", guidance_scale=0, num_inference_steps=4).images[0]

  0%|          | 0/4 [00:00<?, ?it/s]

: 